In [1]:
import requests
import urllib
import json
import lxml.html
from lxml import etree
import logging
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
my_shape = {'MAH':'b01pp0xq',
           'EM' : 'b006wkfp',
           'MAH_HR': 'm0002zgz'}

In [4]:
def get_show_calendar(broadcast_pid,session=session, 
                      api_url='http://clifton.api.bbci.co.uk', 
                      calendar="/aps/programmes/{pid}/episodes.json"):
    '''
    for a given broadcaster returns a dictionary of the years and 
    months a given broadcast show has been on.
    
    Parameters
    ----------
    broadcast_pid : the shows underlying pid
    
    session : Requests session
    
    api_url : bbc api as of 2019
    
    calendar = the api string that gets all the json for the years and months
    
    Returns
    -------
    Dictionary of Year and the Months in the year the show plays
    
    '''
    
    episodes = api_url + calendar.format(pid=broadcast_pid)
    print(episodes)
    all_episodes = session.get(episodes).json()

    hist_mth_yr ={}
    ls = []

    for yr in all_episodes["filters"]["years"]:
    #     print(x["id"])
        mth_list = yr["months"]
        hist_mth_yr[yr['id']] = [mth["id"] for mth in yr['months']]

    return hist_mth_yr

In [5]:
def get_shows_in_mth(broadcast_pid, year, mth,session=session,
                     api_url='http://clifton.api.bbci.co.uk', 
                     api_mth="/aps/programmes/{pid}/episodes/{year}/{month}.json"):
    '''
    Function that takes the Broadcaster/show, Year (int) and Month (int 1=Jan 12=Dec) 
    and returns dictionaty of the show pids with the title
    for a given broadcaster returns a dictionary of the years and 
    months a given broadcast show has been on.
    
    Parameters
    ----------
    broadcast_pid : the shows underlying pid
    year : year
    mth : month
    
    session : Requests session
    
    api_url : bbc api as of 2019
    
    calendar = the api string that gets all the json for the years and months
    
    Returns
    -------
    List 
        0 : full show json
        1 : Dictionary, key episode pid : [episode_date, Broadcaster, show title] 
        3 : DataFrame of the Dictionary
    
    '''
    show_dic = {}
    ep_in_mth_url = api_url + api_mth.format(pid=broadcast_pid,year=year,month=mth)
#     print(ep_in_mth_url)
    show_json = session.get(ep_in_mth_url).json()
    shows = {}

    for x in show_json['broadcasts']:
        pid= x['programme']['pid']
        shows[pid] = {"date" : x['schedule_date'],
                      "broadcast" : x['programme']['programme']['title'],
                      'title' : x['programme']['title'],
                      'duration' : x['programme']['duration'],
                      'station' : x['programme']['ownership']['service']['key']
                     }
    return [show_json,shows,pd.DataFrame(shows).T]
    

In [6]:
def get_episode(episode_pid, session=session,
               api_url='http://clifton.api.bbci.co.uk', 
               api_ep="/aps/programmes/{pid}.json",
               api_ep_tracks="/aps/programmes/{pid}/segments.json" 
               ):
    '''
    Parameters
    ----------
    episode_pid : the shows underlying pid
    
    Returns
    ----------
    
    List of []
    '''
    
    ep_dic = {}
    ep_url = api_url + api_ep.format(pid=episode_pid)
    tracks_url = api_url + api_ep_tracks.format(pid=episode_pid)
#     print(ep_url)
    ep_json = session.get(ep_url).json()
    tracks_json = session.get(tracks_url).json()
    

    episode = {}
    
    x = ep_json['programme']
    episode['pid'] = x['pid']
    episode['short_synopsis'] = x['short_synopsis']
    episode['medium_synopsis'] = x['medium_synopsis']
    episode['long_synopsis'] = x['long_synopsis']              
    episode['image'] = x['image']['pid']

    
    return [ep_json,episode, tracks_json]
    

In [8]:
sj = get_shows_in_mth(my_shape['MAH'],2017,4)

In [9]:
sj[2]

,broadcast,date,duration,station,title
b08nz0qg,Mary Anne Hobbs,2017-04-30,10800,6music,With Mac DeMarco
b08nytml,Mary Anne Hobbs,2017-04-29,10800,6music,With Arrested Development
b08n8qvx,Mary Anne Hobbs,2017-04-23,10800,6music,Iggy Pop's (Extended) Seventieth Birthday Cele...
b08n5t2f,Mary Anne Hobbs,2017-04-22,10800,6music,Celebrating Iggy Pop at 70
b08mkb8y,Mary Anne Hobbs,2017-04-16,10800,6music,Spiritual Sounds
b08mjs3x,Mary Anne Hobbs,2017-04-15,10800,6music,With Fleet Foxes
b08lpxth,Mary Anne Hobbs,2017-04-09,10800,6music,With Perfume Genius
b08lphyd,Mary Anne Hobbs,2017-04-08,10800,6music,Restoring Nina Simone's Home
b08l5p17,Mary Anne Hobbs,2017-04-02,10800,6music,With Anna Meredith
b08l5kyq,Mary Anne Hobbs,2017-04-01,10800,6music,With The Stranglers' JJ Burnel


In [10]:
ep_pid = sj[2].iloc[0].name

In [11]:
ep = get_episode(ep_pid)

In [12]:
ep[2]

{'segment_events': [{'title': None,
   'pid': 'p051k7nz',
   'short_synopsis': '',
   'medium_synopsis': '',
   'long_synopsis': '',
   'version_offset': 21,
   'position': 1,
   'is_chapter': False,
   'has_snippet': False,
   'segment': {'type': 'music',
    'pid': 'p051k7n8',
    'duration': 136,
    'primary_contributor': {'pid': 'p00zzbz3',
     'musicbrainz_gid': '2944824d-4c26-476f-a981-be849081942f',
     'name': 'Nina Simone'},
    'artist': 'Nina Simone',
    'track_title': "I'm Gonna Leave You",
    'track_number': None,
    'publisher': 'Ninandy Music Company',
    'record_label': 'Universal Music (UK) Limited',
    'release_title': 'Wild Is The Wind / High Priestess Of Soul',
    'record_id': 'nzm8zz',
    'catalogue_number': None,
    'contributions': [{'pid': 'p00zzbz3',
      'name': 'Nina Simone',
      'role': 'Performer',
      'musicbrainz_gid': '2944824d-4c26-476f-a981-be849081942f'}],
    'title': "I'm Gonna Leave You",
    'short_synopsis': None,
    'medium_syno

In [27]:
df_tr = pd.DataFrame.from_dict(ep[2]['segment_events'])
df_tr.rename(columns={'pid':"pid_track",
                     'title': 'track_show_special_title'},inplace=True)
# df_tr_temp =pd.DataFrame(df_tr.segment.tolist())
df_tr2  = pd.concat([df_tr,pd.DataFrame(df_tr.segment.tolist())],axis=1)
df_tra = pd.DataFrame(df_tr.segment.tolist())


df_tr2.drop('segment', axis=1, inplace=True)

df_tr2  = pd.concat([df_tr2,pd.DataFrame(df_tr2.primary_contributor.tolist())],axis=1)
df_trb = pd.DataFrame(df_tr2.primary_contributor.tolist())
df_trc = pd.DataFrame(df_tr2.contributions.tolist())

# df_tr2  = pd.concat([df_tr2,pd.DataFrame(df_tr2.primary_contributor.tolist())],axis=1)

In [40]:
df_tr2.primary_contributor.tolist()

[{'pid': 'p00zzbz3',
  'musicbrainz_gid': '2944824d-4c26-476f-a981-be849081942f',
  'name': 'Nina Simone'},
 {'pid': 'p026h7y7',
  'musicbrainz_gid': '09589937-20f0-40f8-a473-b22a0bac9ce5',
  'name': 'Loyle Carner'},
 {'pid': 'p00ykrt2',
  'musicbrainz_gid': '75167b8b-44e4-407b-9d35-effe87b223cf',
  'name': 'Neil Young'},
 {'pid': 'p04c54bh',
  'musicbrainz_gid': 'cf82a38f-9413-4333-bacb-ca5b6db95794',
  'name': 'Warpaint'},
 {'pid': 'p012d9f5',
  'musicbrainz_gid': 'f6f2326f-6b25-4170-b89d-e235b25508e8',
  'name': 'Sigur Rós'},
 {'pid': 'p01kx5pr',
  'musicbrainz_gid': '82c770d1-8c62-4f4d-9021-7bd8a097f7dd',
  'name': 'The Caretaker'},
 {'pid': 'p012d8sn',
  'musicbrainz_gid': 'b6b2bb8d-54a9-491f-9607-7b546023b433',
  'name': 'Pixies'},
 {'pid': 'p01jmsg9',
  'musicbrainz_gid': '209cc789-981e-4911-a5fb-47c30230976f',
  'name': 'Zap Pow'},
 {'pid': 'p012ddn8',
  'musicbrainz_gid': '381086ea-f511-4aba-bdf9-71c753dc5077',
  'name': 'Kendrick Lamar'},
 {'pid': 'p012d8wt',
  'musicbrainz_g

In [28]:
df_tr2.iloc[0]

has_snippet                                                             False
is_chapter                                                              False
long_synopsis                                                                
medium_synopsis                                                              
pid_track                                                            p051k7nz
position                                                                    1
short_synopsis                                                               
track_show_special_title                                                 None
version_offset                                                             21
artist                                                            Nina Simone
catalogue_number                                                         None
contributions               [{'pid': 'p00zzbz3', 'name': 'Nina Simone', 'r...
duration                                                        

In [31]:
df_tra.iloc[0]

artist                                                       Nina Simone
catalogue_number                                                    None
contributions          [{'pid': 'p00zzbz3', 'name': 'Nina Simone', 'r...
duration                                                             136
long_synopsis                                                       None
medium_synopsis                                                     None
pid                                                             p051k7n8
primary_contributor    {'pid': 'p00zzbz3', 'musicbrainz_gid': '294482...
publisher                                          Ninandy Music Company
record_id                                                         nzm8zz
record_label                                Universal Music (UK) Limited
release_title                  Wild Is The Wind / High Priestess Of Soul
short_synopsis                                                      None
title                                              

In [30]:
df_trb.iloc[0]

musicbrainz_gid    2944824d-4c26-476f-a981-be849081942f
name                                        Nina Simone
pid                                            p00zzbz3
Name: 0, dtype: object

In [38]:
for x in range(60):
    print(df_trc.iloc[x][0]['role'])

Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer
Performer


IndexError: single positional indexer is out-of-bounds

In [279]:
df_tr2.columns

Index(['has_snippet', 'is_chapter', 'long_synopsis', 'medium_synopsis',
       'pid_track', 'position', 'segment', 'short_synopsis', 'title',
       'version_offset', 'artist', 'catalogue_number', 'contributions',
       'duration', 'long_synopsis', 'medium_synopsis', 'pid',
       'primary_contributor', 'publisher', 'record_id', 'record_label',
       'release_title', 'short_synopsis', 'title', 'track_number',
       'track_title', 'type'],
      dtype='object')

In [282]:
df_tr2.loc[0]['segment']

{'type': 'music',
 'pid': 'p051k7n8',
 'duration': 136,
 'primary_contributor': {'pid': 'p00zzbz3',
  'musicbrainz_gid': '2944824d-4c26-476f-a981-be849081942f',
  'name': 'Nina Simone'},
 'artist': 'Nina Simone',
 'track_title': "I'm Gonna Leave You",
 'track_number': None,
 'publisher': 'Ninandy Music Company',
 'record_label': 'Universal Music (UK) Limited',
 'release_title': 'Wild Is The Wind / High Priestess Of Soul',
 'record_id': 'nzm8zz',
 'catalogue_number': None,
 'contributions': [{'pid': 'p00zzbz3',
   'name': 'Nina Simone',
   'role': 'Performer',
   'musicbrainz_gid': '2944824d-4c26-476f-a981-be849081942f'}],
 'title': "I'm Gonna Leave You",
 'short_synopsis': None,
 'medium_synopsis': None,
 'long_synopsis': None}

In [235]:
ep[1]

{'pid': 'b08nz0qg',
 'short_synopsis': 'Mary Anne catches up with Mac DeMarco ahead of the release of his new album This Old Dog.',
 'medium_synopsis': 'Mary Anne catches up with Mac DeMarco to talk about his new album and his search for an assistant for his fan club...',
 'long_synopsis': "Mary Anne catches up with Canadian singer-songwriter Mac DeMarco, ahead of the release of his new album This Old Dog.\n\nAs the search for an assistant for his fan club goes on, we also find out more about DeMarco's special relationship with his fans, and why he feels it is important to connect with them.\n\n'This Old Dog' will be the singer's third album, and is due for release in May 2017. Expect tunes from this, plus music from those who have influenced DeMarco's sound, including Neil Young and Steely Dan.",
 'image': 'p051df8r'}

In [225]:
sj[1]

{'b08nz0qg': {'date': '2017-04-30',
  'broadcast': 'Mary Anne Hobbs',
  'title': 'With Mac DeMarco',
  'duration': 10800,
  'station': '6music'},
 'b08nytml': {'date': '2017-04-29',
  'broadcast': 'Mary Anne Hobbs',
  'title': 'With Arrested Development',
  'duration': 10800,
  'station': '6music'},
 'b08n8qvx': {'date': '2017-04-23',
  'broadcast': 'Mary Anne Hobbs',
  'title': "Iggy Pop's (Extended) Seventieth Birthday Celebrations",
  'duration': 10800,
  'station': '6music'},
 'b08n5t2f': {'date': '2017-04-22',
  'broadcast': 'Mary Anne Hobbs',
  'title': 'Celebrating Iggy Pop at 70',
  'duration': 10800,
  'station': '6music'},
 'b08mkb8y': {'date': '2017-04-16',
  'broadcast': 'Mary Anne Hobbs',
  'title': 'Spiritual Sounds',
  'duration': 10800,
  'station': '6music'},
 'b08mjs3x': {'date': '2017-04-15',
  'broadcast': 'Mary Anne Hobbs',
  'title': 'With Fleet Foxes',
  'duration': 10800,
  'station': '6music'},
 'b08lpxth': {'date': '2017-04-09',
  'broadcast': 'Mary Anne Hobbs

In [196]:
sj[0]

{'broadcasts': [{'is_repeat': False,
   'is_blanked': False,
   'schedule_date': '2017-04-30',
   'start': '2017-04-30T07:00:00+01:00',
   'end': '2017-04-30T10:00:00+01:00',
   'duration': 10800,
   'service': {'type': 'radio',
    'id': 'bbc_6music',
    'key': '6music',
    'title': 'BBC Radio 6 Music'},
   'programme': {'type': 'episode',
    'pid': 'b08nz0qg',
    'position': None,
    'title': 'With Mac DeMarco',
    'short_synopsis': 'Mary Anne catches up with Mac DeMarco ahead of the release of his new album This Old Dog.',
    'media_type': 'audio',
    'duration': 10800,
    'image': {'pid': 'p051df8r'},
    'display_titles': {'title': 'Mary Anne Hobbs',
     'subtitle': 'With Mac DeMarco'},
    'first_broadcast_date': '2017-04-30T07:00:00+01:00',
    'ownership': {'service': {'type': 'radio',
      'id': 'bbc_6music',
      'key': '6music',
      'title': 'BBC Radio 6 Music'}},
    'programme': {'type': 'brand',
     'pid': 'b01pp0xq',
     'title': 'Mary Anne Hobbs',
     '

In [195]:
ep_get = list(sj[1].keys())[0]
ep_get

'b08nz0qg'

In [217]:
ep = get_episode(ep_get)

In [237]:
sj[0]

{'broadcasts': [{'is_repeat': False,
   'is_blanked': False,
   'schedule_date': '2017-04-30',
   'start': '2017-04-30T07:00:00+01:00',
   'end': '2017-04-30T10:00:00+01:00',
   'duration': 10800,
   'service': {'type': 'radio',
    'id': 'bbc_6music',
    'key': '6music',
    'title': 'BBC Radio 6 Music'},
   'programme': {'type': 'episode',
    'pid': 'b08nz0qg',
    'position': None,
    'title': 'With Mac DeMarco',
    'short_synopsis': 'Mary Anne catches up with Mac DeMarco ahead of the release of his new album This Old Dog.',
    'media_type': 'audio',
    'duration': 10800,
    'image': {'pid': 'p051df8r'},
    'display_titles': {'title': 'Mary Anne Hobbs',
     'subtitle': 'With Mac DeMarco'},
    'first_broadcast_date': '2017-04-30T07:00:00+01:00',
    'ownership': {'service': {'type': 'radio',
      'id': 'bbc_6music',
      'key': '6music',
      'title': 'BBC Radio 6 Music'}},
    'programme': {'type': 'brand',
     'pid': 'b01pp0xq',
     'title': 'Mary Anne Hobbs',
     '

In [232]:
s = pd.DataFrame(sj[1])

In [234]:
s.T

,broadcast,date,duration,station,title
b08nz0qg,Mary Anne Hobbs,2017-04-30,10800,6music,With Mac DeMarco
b08nytml,Mary Anne Hobbs,2017-04-29,10800,6music,With Arrested Development
b08n8qvx,Mary Anne Hobbs,2017-04-23,10800,6music,Iggy Pop's (Extended) Seventieth Birthday Cele...
b08n5t2f,Mary Anne Hobbs,2017-04-22,10800,6music,Celebrating Iggy Pop at 70
b08mkb8y,Mary Anne Hobbs,2017-04-16,10800,6music,Spiritual Sounds
b08mjs3x,Mary Anne Hobbs,2017-04-15,10800,6music,With Fleet Foxes
b08lpxth,Mary Anne Hobbs,2017-04-09,10800,6music,With Perfume Genius
b08lphyd,Mary Anne Hobbs,2017-04-08,10800,6music,Restoring Nina Simone's Home
b08l5p17,Mary Anne Hobbs,2017-04-02,10800,6music,With Anna Meredith
b08l5kyq,Mary Anne Hobbs,2017-04-01,10800,6music,With The Stranglers' JJ Burnel


In [26]:
show = {}
shows={}
for x in sj['broadcasts']:
    pid= x['programme']['pid']
    show[pid] = [x['programme']['pid']
        
    ]
#     print(x['programme']['pid'])
    



In [146]:
ep = get_episode("b08nz0qg")

In [148]:
ep[0]['programme']

{'type': 'episode',
 'pid': 'b08nz0qg',
 'expected_child_count': None,
 'position': None,
 'image': {'pid': 'p051df8r'},
 'media_type': 'audio',
 'title': 'With Mac DeMarco',
 'short_synopsis': 'Mary Anne catches up with Mac DeMarco ahead of the release of his new album This Old Dog.',
 'medium_synopsis': 'Mary Anne catches up with Mac DeMarco to talk about his new album and his search for an assistant for his fan club...',
 'long_synopsis': "Mary Anne catches up with Canadian singer-songwriter Mac DeMarco, ahead of the release of his new album This Old Dog.\n\nAs the search for an assistant for his fan club goes on, we also find out more about DeMarco's special relationship with his fans, and why he feels it is important to connect with them.\n\n'This Old Dog' will be the singer's third album, and is due for release in May 2017. Expect tunes from this, plus music from those who have influenced DeMarco's sound, including Neil Young and Steely Dan.",
 'first_broadcast_date': '2017-04-30

In [82]:
df_ep = pd.DataFrame.from_dict(ep[1]["segment_events"])

In [83]:
df_ep

,has_snippet,is_chapter,long_synopsis,medium_synopsis,pid,position,segment,short_synopsis,title,version_offset
0,False,False,,,p051k7nz,1,"{'type': 'music', 'pid': 'p051k7n8', 'duration...",,None,21
1,False,False,,,p051k7xx,2,"{'type': 'music', 'pid': 'p04y9lhq', 'duration...",,None,157
2,False,False,,,p051k8br,3,"{'type': 'music', 'pid': 'p01dc9m8', 'duration...",,None,396
3,False,False,,,p051k8jg,4,"{'type': 'music', 'pid': 'p05008rk', 'duration...",,None,646
4,False,False,,,p051k8nw,5,"{'type': 'music', 'pid': 'p051k8nv', 'duration...",,None,932
5,False,False,,,p051k8px,6,"{'type': 'music', 'pid': 'p051k8pw', 'duration...",,None,1270
6,False,False,,,p051k8q6,7,"{'type': 'music', 'pid': 'p03syz9t', 'duration...",,None,1552
7,False,False,,,p051k8w9,8,"{'type': 'music', 'pid': 'p050jlds', 'duration...",,None,1959
8,False,False,,,p051k8zh,9,"{'type': 'music', 'pid': 'p051k8zg', 'duration...",,None,2141
9,False,False,,,p051k8zr,10,"{'type': 'music', 'pid': 'p04mv04s', 'duration...",,None,2362


In [86]:
df_ep2  = pd.concat([df_ep,pd.DataFrame(df_ep.segment.tolist())],axis=1)


In [87]:
df_ep2

,has_snippet,is_chapter,long_synopsis,medium_synopsis,pid,position,segment,short_synopsis,title,version_offset,...,primary_contributor,publisher,record_id,record_label,release_title,short_synopsis,title,track_number,track_title,type
0,False,False,,,p051k7nz,1,"{'type': 'music', 'pid': 'p051k7n8', 'duration...",,None,21,...,"{'pid': 'p00zzbz3', 'musicbrainz_gid': '294482...",Ninandy Music Company,nzm8zz,Universal Music (UK) Limited,Wild Is The Wind / High Priestess Of Soul,None,I'm Gonna Leave You,None,I'm Gonna Leave You,music
1,False,False,,,p051k7xx,2,"{'type': 'music', 'pid': 'p04y9lhq', 'duration...",,None,157,...,"{'pid': 'p026h7y7', 'musicbrainz_gid': '095899...",Copyright Control/Liuto Edizioni Musicali di S...,n3x8hc,AMF Records,None,None,Ain't Nothing Changed,None,Ain't Nothing Changed,music
2,False,False,,,p051k8br,3,"{'type': 'music', 'pid': 'p01dc9m8', 'duration...",,None,396,...,"{'pid': 'p00ykrt2', 'musicbrainz_gid': '75167b...",SILVER FIDDLE MUSIC ASCAP,nz2cn6,REPRISE,None,None,Harvest Moon,None,Harvest Moon,music
3,False,False,,,p051k8jg,4,"{'type': 'music', 'pid': 'p05008rk', 'duration...",,None,646,...,"{'pid': 'p04c54bh', 'musicbrainz_gid': 'cf82a3...",qqpp music,n43jnz,None,None,None,The Stall,None,The Stall,music
4,False,False,,,p051k8nw,5,"{'type': 'music', 'pid': 'p051k8nv', 'duration...",,None,932,...,"{'pid': 'p012d9f5', 'musicbrainz_gid': 'f6f232...",None,nzvw3q,None,None,None,"Stormur (6 Music Session, 18 Oct 2013)",None,"Stormur (6 Music Session, 18 Oct 2013)",music
5,False,False,,,p051k8px,6,"{'type': 'music', 'pid': 'p051k8pw', 'duration...",,None,1270,...,"{'pid': 'p01kx5pr', 'musicbrainz_gid': '82c770...",None,n4rw8v,(History Always Favours The Winners,None,None,Glimpses Of Hope In Trying Times,None,Glimpses Of Hope In Trying Times,music
6,False,False,,,p051k8q6,7,"{'type': 'music', 'pid': 'p03syz9t', 'duration...",,None,1552,...,"{'pid': 'p012d8sn', 'musicbrainz_gid': 'b6b2bb...",RICE AND BEANS MUSIC,nz32b4,4 A.D.,None,None,Where Is My Mind?,None,Where Is My Mind?,music
7,False,False,,,p051k8w9,8,"{'type': 'music', 'pid': 'p050jlds', 'duration...",,None,1959,...,"{'pid': 'p01jmsg9', 'musicbrainz_gid': '209cc7...",Universal/Island / Upstairs Music,nz325v,Island,None,None,This Is Reggae Music,None,This Is Reggae Music,music
8,False,False,,,p051k8zh,9,"{'type': 'music', 'pid': 'p051k8zg', 'duration...",,None,2141,...,"{'pid': 'p012ddn8', 'musicbrainz_gid': '381086...",Aftermath/Interscope (Top Dawg Entertainment),n4rx9g,Polydor,None,None,YAH.,None,YAH.,music
9,False,False,,,p051k8zr,10,"{'type': 'music', 'pid': 'p04mv04s', 'duration...",,None,2362,...,"{'pid': 'p012d8wt', 'musicbrainz_gid': 'ba2f4f...",Underworld,nzzvqw,Junior Boys Own,None,None,Born Slippy,None,Born Slippy,music


In [25]:
sj

{'broadcasts': [{'is_repeat': False,
   'is_blanked': False,
   'schedule_date': '2017-04-30',
   'start': '2017-04-30T07:00:00+01:00',
   'end': '2017-04-30T10:00:00+01:00',
   'duration': 10800,
   'service': {'type': 'radio',
    'id': 'bbc_6music',
    'key': '6music',
    'title': 'BBC Radio 6 Music'},
   'programme': {'type': 'episode',
    'pid': 'b08nz0qg',
    'position': None,
    'title': 'With Mac DeMarco',
    'short_synopsis': 'Mary Anne catches up with Mac DeMarco ahead of the release of his new album This Old Dog.',
    'media_type': 'audio',
    'duration': 10800,
    'image': {'pid': 'p051df8r'},
    'display_titles': {'title': 'Mary Anne Hobbs',
     'subtitle': 'With Mac DeMarco'},
    'first_broadcast_date': '2017-04-30T07:00:00+01:00',
    'ownership': {'service': {'type': 'radio',
      'id': 'bbc_6music',
      'key': '6music',
      'title': 'BBC Radio 6 Music'}},
    'programme': {'type': 'brand',
     'pid': 'b01pp0xq',
     'title': 'Mary Anne Hobbs',
     '

In [92]:
sj.requests.get('items', None)

AttributeError: 'dict' object has no attribute 'requests'

In [54]:
10800/60

180.0

In [46]:
df = pd.DataFrame.from_dict(sj['broadcasts'])

In [52]:
df.columns

Index(['duration', 'end', 'is_blanked', 'is_repeat', 'programme',
       'schedule_date', 'service', 'start'],
      dtype='object')

In [53]:
pd.DataFrame(df.programme.tolist())

,display_titles,duration,first_broadcast_date,image,is_available_mediaset_pc_sd,is_legacy_media,media_type,ownership,pid,position,programme,short_synopsis,title,type
0,"{'title': 'Mary Anne Hobbs', 'subtitle': 'With...",10800,2017-04-30T07:00:00+01:00,{'pid': 'p051df8r'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08nz0qg,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne catches up with Mac DeMarco ahead of...,With Mac DeMarco,episode
1,"{'title': 'Mary Anne Hobbs', 'subtitle': 'With...",10800,2017-04-29T07:00:00+01:00,{'pid': 'p050tgk8'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08nytml,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne talks to Speech from Arrested Develo...,With Arrested Development,episode
2,"{'title': 'Mary Anne Hobbs', 'subtitle': 'Iggy...",10800,2017-04-23T07:00:00+01:00,{'pid': 'p050lmmq'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08n8qvx,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne Hobbs continues her chat with Iggy Pop.,Iggy Pop's (Extended) Seventieth Birthday Cele...,episode
3,"{'title': 'Mary Anne Hobbs', 'subtitle': 'Cele...",10800,2017-04-22T07:00:00+01:00,{'pid': 'p050dx0p'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08n5t2f,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...","Mary Anne Hobbs is joined by Iggy Pop, the mor...",Celebrating Iggy Pop at 70,episode
4,"{'title': 'Mary Anne Hobbs', 'subtitle': 'Spir...",10800,2017-04-16T07:00:00+01:00,{'pid': 'p04zvqbd'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08mkb8y,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne wants your gospel-inspired tunes for...,Spiritual Sounds,episode
5,"{'title': 'Mary Anne Hobbs', 'subtitle': 'With...",10800,2017-04-15T07:00:00+01:00,{'pid': 'p04zn9zv'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08mjs3x,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne speaks to Robin Pecknold of Fleet Fo...,With Fleet Foxes,episode
6,"{'title': 'Mary Anne Hobbs', 'subtitle': 'With...",10800,2017-04-09T07:00:00+01:00,{'pid': 'p04ymtdw'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08lpxth,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne is joined by Perfume Genius aka the ...,With Perfume Genius,episode
7,"{'title': 'Mary Anne Hobbs', 'subtitle': 'Rest...",10800,2017-04-08T07:00:00+01:00,{'pid': 'p04z1zg4'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08lphyd,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne speaks to two artists who have just ...,Restoring Nina Simone's Home,episode
8,"{'title': 'Mary Anne Hobbs', 'subtitle': 'With...",10800,2017-04-02T07:00:00+01:00,{'pid': 'p04yjjkl'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08l5p17,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne Hobbs brings you her highlights from...,With Anna Meredith,episode
9,"{'title': 'Mary Anne Hobbs', 'subtitle': 'With...",10800,2017-04-01T07:00:00+01:00,{'pid': 'p04yjhpn'},False,False,audio,"{'service': {'type': 'radio', 'id': 'bbc_6musi...",b08l5kyq,None,"{'type': 'brand', 'pid': 'b01pp0xq', 'title': ...",Mary Anne Hobbs poses six burning questions to...,With The Stranglers' JJ Burnel,episode


In [42]:
df2  = pd.concat([df,pd.DataFrame(df.programme.tolist())],axis=1)

In [48]:
df2  = pd.concat([df,df.programme.apply(pd.Series)],axis=1)

In [51]:
df2['programme']

,programme,programme
0,"{'type': 'episode', 'pid': 'b08nz0qg', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
1,"{'type': 'episode', 'pid': 'b08nytml', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
2,"{'type': 'episode', 'pid': 'b08n8qvx', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
3,"{'type': 'episode', 'pid': 'b08n5t2f', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
4,"{'type': 'episode', 'pid': 'b08mkb8y', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
5,"{'type': 'episode', 'pid': 'b08mjs3x', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
6,"{'type': 'episode', 'pid': 'b08lpxth', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
7,"{'type': 'episode', 'pid': 'b08lphyd', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
8,"{'type': 'episode', 'pid': 'b08l5p17', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."
9,"{'type': 'episode', 'pid': 'b08l5kyq', 'positi...","{'type': 'brand', 'pid': 'b01pp0xq', 'title': ..."


In [2]:
''' http://clifton.api.bbci.co.uk/

# with URLs matching the following routes

/aps/programmes/{pid}/episodes/{year}/{month}.json
/aps/programmes/{pid}/episodes/last.json
/aps/programmes/{pid}/episodes/upcoming.json
/aps/programmes/{pid}/episodes.json
/aps/programmes/{pid}/children.json
/aps/programmes/{pid}/segments.json
/aps/programmes/{pid}/series.json
/aps/programmes/{pid}.json
/aps/programmes/{categoryType}.json
/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/upcoming.json
/aps/programmes/{categoryType}/{urlKeyHierarchy}/player/episodes.json
/aps/programmes/{categoryType}/{urlKeyHierarchy}/{slice}.json
/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/{year}/{month}/calendar.json
/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/{year}/{month}/{day}/subcategories.json
/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/{year}/{month}/{day}.json
/aps/programmes/{categoryType}/{urlKeyHierarchy}.json
/aps/{networkKey}/programmes/music/artists/charts.json
/aps/programmes/music/artists/charts.json
/aps/programmes/music/artists/{musicBrainzId}.json
/aps/programmes/a-z.json
/aps/programmes/a-z/{slice}.json
/aps/programmes/a-z/by/{search}.json
/aps/programmes/a-z/by/{search}/{slice}.json
/availability/{pid}.json
/
/debug/{serviceName}/{serviceMethod}

'''

' http://clifton.api.bbci.co.uk/\n\n# with URLs matching the following routes\n\n/aps/programmes/{pid}/episodes/{year}/{month}.json\n/aps/programmes/{pid}/episodes/last.json\n/aps/programmes/{pid}/episodes/upcoming.json\n/aps/programmes/{pid}/episodes.json\n/aps/programmes/{pid}/children.json\n/aps/programmes/{pid}/segments.json\n/aps/programmes/{pid}/series.json\n/aps/programmes/{pid}.json\n/aps/programmes/{categoryType}.json\n/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/upcoming.json\n/aps/programmes/{categoryType}/{urlKeyHierarchy}/player/episodes.json\n/aps/programmes/{categoryType}/{urlKeyHierarchy}/{slice}.json\n/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/{year}/{month}/calendar.json\n/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/{year}/{month}/{day}/subcategories.json\n/aps/programmes/{categoryType}/{urlKeyHierarchy}/schedules/{year}/{month}/{day}.json\n/aps/programmes/{categoryType}/{urlKeyHierarchy}.json\n/aps/{networkKey}/programmes/m

In [3]:
url = 'http://clifton.api.bbci.co.uk'
pid ='b09y1xkp'
MAH = 'b01pp0xq'
segments = "/aps/programmes/{pid}/segments.json".format(pid=pid)
episodes = "/aps/programmes/{pid}/episodes.json".format(pid=MAH)



# MAH = 'https://www.bbc.co.uk/programmes/b01pp0xq/episodes/player'



In [4]:
segments

'/aps/programmes/b09y1xkp/segments.json'

In [94]:
url_get = url + segments
url_ep = url + episodes

In [108]:
session = requests.Session()
pg = session.get(url_get)

In [106]:
from requests import get

In [109]:
pg.raise_for_status()

In [120]:
x = pg.json().get('segment_events', None)

In [121]:
x

[{'title': None,
  'pid': 'p0651fmp',
  'short_synopsis': '',
  'medium_synopsis': '',
  'long_synopsis': '',
  'version_offset': 18,
  'position': 1,
  'is_chapter': False,
  'has_snippet': False,
  'segment': {'type': 'music',
   'pid': 'p0651fmn',
   'duration': 163,
   'primary_contributor': {'pid': 'p012dw7p',
    'musicbrainz_gid': 'fb5d53ea-6090-49cd-b8ab-0d200bacf4c3',
    'name': 'Dobie Gray'},
   'artist': 'Dobie Gray',
   'track_title': 'The In Crowd',
   'track_number': None,
   'publisher': 'Cross Music Limited',
   'record_label': 'Columbia',
   'release_title': 'Originals 2  (Various Artists)',
   'record_id': 'nz22vg',
   'catalogue_number': None,
   'contributions': [{'pid': 'p012dw7p',
     'name': 'Dobie Gray',
     'role': 'Performer',
     'musicbrainz_gid': 'fb5d53ea-6090-49cd-b8ab-0d200bacf4c3'}],
   'title': 'The In Crowd',
   'short_synopsis': None,
   'medium_synopsis': None,
   'long_synopsis': None}},
 {'title': None,
  'pid': 'p0651fyr',
  'short_synopsis':

In [118]:
pg.json()

{'segment_events': [{'title': None,
   'pid': 'p0651fmp',
   'short_synopsis': '',
   'medium_synopsis': '',
   'long_synopsis': '',
   'version_offset': 18,
   'position': 1,
   'is_chapter': False,
   'has_snippet': False,
   'segment': {'type': 'music',
    'pid': 'p0651fmn',
    'duration': 163,
    'primary_contributor': {'pid': 'p012dw7p',
     'musicbrainz_gid': 'fb5d53ea-6090-49cd-b8ab-0d200bacf4c3',
     'name': 'Dobie Gray'},
    'artist': 'Dobie Gray',
    'track_title': 'The In Crowd',
    'track_number': None,
    'publisher': 'Cross Music Limited',
    'record_label': 'Columbia',
    'release_title': 'Originals 2  (Various Artists)',
    'record_id': 'nz22vg',
    'catalogue_number': None,
    'contributions': [{'pid': 'p012dw7p',
      'name': 'Dobie Gray',
      'role': 'Performer',
      'musicbrainz_gid': 'fb5d53ea-6090-49cd-b8ab-0d200bacf4c3'}],
    'title': 'The In Crowd',
    'short_synopsis': None,
    'medium_synopsis': None,
    'long_synopsis': None}},
  {'titl

In [12]:
ep_json = ep.json()

In [23]:
all_episodes = session.get(url_ep).json()

hist_mth_yr ={}
ls = []

for yr in all_episodes["filters"]["years"]:
#     print(x["id"])
    mth_list = yr["months"]
    hist_mth_yr[yr['id']] = [mth["id"] for mth in yr['months']]


In [39]:
print(get_show_calendar(MAH))

http://clifton.api.bbci.co.uk/aps/programmes/b01pp0xq/episodes.json
{2019: [7, 6, 5, 4, 3, 2, 1], 2018: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2017: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2016: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2015: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2014: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2013: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]}


In [40]:

print(get_show_calendar(EM))

http://clifton.api.bbci.co.uk/aps/programmes/b006wkfp/episodes.json
{2019: [7, 6, 5, 4, 3, 2, 1], 2018: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2017: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2016: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2015: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2014: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2013: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2012: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2011: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2010: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2009: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2008: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2007: [12, 11, 10, 9]}


In [54]:
sj = get_shows_in_mth(my_shape["MAH"],2018,3)

http://clifton.api.bbci.co.uk/aps/programmes/b01pp0xq/episodes/2018/3.json


In [57]:
sj

{'broadcasts': [{'is_repeat': False,
   'is_blanked': False,
   'schedule_date': '2018-03-31',
   'start': '2018-03-31T07:00:00+01:00',
   'end': '2018-03-31T10:00:00+01:00',
   'duration': 10800,
   'service': {'type': 'radio',
    'id': 'bbc_6music',
    'key': '6music',
    'title': 'BBC Radio 6 Music'},
   'programme': {'type': 'episode',
    'pid': 'b09x8ckx',
    'position': None,
    'title': 'With Braille Face',
    'short_synopsis': 'Mary Anne is joined by Melbourne-based musician Jordan White aka Braille Face.',
    'media_type': 'audio',
    'duration': 10800,
    'image': {'pid': 'p0623d2q'},
    'display_titles': {'title': 'Mary Anne Hobbs',
     'subtitle': 'With Braille Face'},
    'first_broadcast_date': '2018-03-31T07:00:00+01:00',
    'ownership': {'service': {'type': 'radio',
      'id': 'bbc_6music',
      'key': '6music',
      'title': 'BBC Radio 6 Music'}},
    'programme': {'type': 'brand',
     'pid': 'b01pp0xq',
     'title': 'Mary Anne Hobbs',
     'position'

In [15]:
ep_in_mth = url + "/aps/programmes/{pid}/episodes/{year}/{month}.json".format(pid=MAH,year=2019,month=1)

In [16]:
eps = session.get(ep_in_mth)

In [18]:
epx = eps.json()

In [19]:
for e in  epx['broadcasts']:
    print(e['programme'])


{'type': 'episode', 'pid': 'm0002799', 'position': None, 'title': '31/01/2019', 'short_synopsis': 'Fascinating musical journeys with an infusion of the Arts, mid-mornings for music lovers.', 'media_type': 'audio', 'duration': 8880, 'image': {'pid': 'p06x0hxb'}, 'display_titles': {'title': 'Mary Anne Hobbs', 'subtitle': '31/01/2019'}, 'first_broadcast_date': '2019-01-31T10:30:00Z', 'programme': {'type': 'brand', 'pid': 'b01pp0xq', 'title': 'Mary Anne Hobbs', 'position': None, 'image': {'pid': 'p06x0hxb'}, 'expected_child_count': None, 'first_broadcast_date': '2013-01-05T07:00:00Z', 'ownership': {'service': {'type': 'radio', 'id': 'bbc_6music', 'key': '6music', 'title': 'BBC Radio 6 Music'}}}, 'is_available_mediaset_pc_sd': False, 'is_legacy_media': False}
{'type': 'episode', 'pid': 'm0002781', 'position': None, 'title': 'With Reuben Dangoor', 'short_synopsis': 'Mary Anne has a Three Minute Epiphany from the artist Reuben Dangoor.', 'media_type': 'audio', 'duration': 8880, 'image': {'pid

In [ ]:
url = 'https://www.bbc.co.uk/programmes/'
end ='b09y1xkp'


MAH = 'https://www.bbc.co.uk/programmes/b01pp0xq/episodes/player'



# These two lines enable debugging at httplib level (requests->urllib3->http.client)
# You will see the REQUEST, including HEADERS and DATA, and RESPONSE with HEADERS but without DATA.
# The only thing missing will be the response.body which is not logged.
try:
    import http.client as http_client
except ImportError:
    # Python 2
    import httplib as http_client
http_client.HTTPConnection.debuglevel = 1

# You must initialize logging, otherwise you'll not see debug output.
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True


url_get = url + end
url_get_segments = url_get + '/segments.inc'
url_get_segments_json = url_get + '/segments.json'

session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'})
html = session.get(url_get)
# seg1 = session.get(url_get_segments)

sessionCookies = html.cookies

# seg_json = session.get(url_get_segments_json, cookies = sessionCookies)

In [ ]:
sessionCookies.get_dict()

In [ ]:
response = session.get('http://google.com')
print(session.cookies.get_dict())

In [ ]:

session = requests.Session()
html = session.get(url_get)
session.cookies

In [ ]:
session.headers

In [ ]:
html = session.get(url_get)

In [ ]:
print(session.cookies.get_dict())

In [ ]:
seg_json = session.get(url_get_segments_json)

In [ ]:
for x in session.cookies:
    print(x)

In [ ]:
url_get_segments_json

In [ ]:
session.cookies.update({
  'ptimizelyEndUserId':'oeu1551249247507r0.812459418636283',
    'ckns_sa_labels_persist':"",
    'ckns_orb_fig_cache':"%22ad%22:1%2C%22ap%22:16%2C%22ck%22:0%2C%22eu%22:0%2C%22uk%22:0",
    'ckns_policy':"111",
    '_em_vt':'ac5f6e7d-f1c5-42f3-a0b3-fd3c20b80b66-1692da97858-ac125950; ecos.dt=1559784164690; atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2283530ecc-732e-47d5-bf9c-0c95aace96d7%22%2C%22options%22%3A%7B%22end%22%3A%222020-07-07T12%3A58%3A59.073Z%22%2C%22path%22%3A%22%2F%22%7D%7D'
})

In [ ]:
session.cookies.update({
        'optimizelyEndUserId': 'oeu1551249247507r0.812459418636283',
    })

In [ ]:
headers = {
    'accept' :'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'upgrade-insecure-requests': '1',
    'desktop':"1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
}

In [ ]:
seg_json = session.get(url_get_segments_json,headers=headers)


In [ ]:
print(session.cookies.get_dict())

In [ ]:
session.cookie = 'optimizelyEndUserId=oeu1551249247507r0.812459418636283; ckns_sa_labels_persist={}; ckns_orb_fig_cache={%22ad%22:1%2C%22ap%22:16%2C%22ck%22:0%2C%22eu%22:0%2C%22uk%22:0}; ckns_policy=111; _em_vt=ac5f6e7d-f1c5-42f3-a0b3-fd3c20b80b66-1692da97858-ac125950; ecos.dt=1559784164690; atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2283530ecc-732e-47d5-bf9c-0c95aace96d7%22%2C%22options%22%3A%7B%22end%22%3A%222020-07-07T12%3A58%3A59.073Z%22%2C%22path%22%3A%22%2F%22%7D%7D'

In [ ]:
seg_json.text

In [ ]:
html.cookies

In [ ]:
my_cookies = {'_em_vt': 'ac5f6e7d-f1c5-42f3-a0b3-fd3c20b80b66-1692da97858-ac125950',
              'atuserid': '%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2283530ecc-732e-47d5-bf9c-0c95aace96d7%22%2C%22options%22%3A%7B%22end%22%3A%222020-07-07T12%3A58%3A59.073Z%22%2C%22path%22%3A%22%2F%22%7D%7D',
              'ckns_orb_fig_cache':'{%22ad%22:1%2C%22ap%22:16%2C%22ck%22:0%2C%22eu%22:0%2C%22uk%22:0}',
                'ckns_policy':'111',
             'ckns_sa_labels_persist':'',
             'ecos.dt': '1559784164690',
             'optimizelyEndUserId':'oeu1551249247507r0.812459418636283'}
response = requests.get(
    'https://www.bbc.co.uk/programmes/b09y1wkb/segments.json',
#     'https://www.bbc.co.uk/programmes/b0b11d3n/segments.json',
    cookies=my_cookies)

In [ ]:
# my_cookies = {'optimizelyEndUserId':''}
response = requests.get(
    'https://www.bbc.co.uk/programmes/b0b11d3n/segments.json',)
#     cookies=my_cookies)

In [ ]:
response.text

In [ ]:
response.text

In [ ]:
'https://www.bbc.co.uk/programmes/b0b11d3n'
    
my_cookies = {'optimizelyEndUserId':''}
response = requests.get(
    'https://www.bbc.co.uk/programmes/b0b11d3n/segments.json',
    cookies=my_cookies)

In [ ]:
response

In [47]:
import requests

session = requests.Session()
my_cookies = {'_em_vt': '',
              'atuserid': '',
              'ckns_orb_fig_cache':'',
                'ckns_policy':'',
             'ckns_sa_labels_persist':'',
             'ecos.dt': '',
             'optimizelyEndUserId':''}
headers = {
    "upgrade-insecure-requests": '1',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
}

In [35]:
response2 = session.get(
    'https://www.bbc.co.uk/programmes/b09y1wkb/segments.json',
#     'https://www.bbc.co.uk/programmes/b0b11d3n/segments.json',
    cookies=my_cookies,headers=headers)

In [63]:
response3 = session.get(
#     'https://www.bbc.co.uk/programmes/b09y1wkb/segments.json',
#     'https://www.bbc.co.uk/programmes/b0b11c0n/segments.inc',
#     'https://www.bbc.co.uk/programmes/m0005l7w/segments.inc',
    'https://www.bbc.co.uk/programmes/m0005l7w/segments.json',
    headers=headers)

In [41]:
response.text

'{"segment_events":[{"title":null,"pid":"p0651fmp","short_synopsis":"","medium_synopsis":"","long_synopsis":"","version_offset":18,"position":1,"is_chapter":false,"has_snippet":false,"segment":{"type":"music","pid":"p0651fmn","duration":163,"primary_contributor":{"pid":"p012dw7p","musicbrainz_gid":"fb5d53ea-6090-49cd-b8ab-0d200bacf4c3","name":"Dobie Gray"},"artist":"Dobie Gray","track_title":"The In Crowd","track_number":null,"publisher":"Cross Music Limited","record_label":"Columbia","release_title":"Originals 2  (Various Artists)","record_id":"nz22vg","catalogue_number":null,"contributions":[{"pid":"p012dw7p","name":"Dobie Gray","role":"Performer","musicbrainz_gid":"fb5d53ea-6090-49cd-b8ab-0d200bacf4c3"}],"title":"The In Crowd","short_synopsis":null,"medium_synopsis":null,"long_synopsis":null}},{"title":null,"pid":"p0651fyr","short_synopsis":"","medium_synopsis":"","long_synopsis":"","version_offset":180,"position":2,"is_chapter":false,"has_snippet":false,"segment":{"type":"music","p

In [5]:
print(session.cookies.get_dict())

{}


In [43]:
response2.text

'{"segment_events":[{"title":null,"pid":"p065l7t9","short_synopsis":"","medium_synopsis":"","long_synopsis":"","version_offset":13,"position":1,"is_chapter":false,"has_snippet":false,"segment":{"type":"music","pid":"p065l7t3","duration":240,"primary_contributor":{"pid":"p012dc44","musicbrainz_gid":"63aa26c3-d59b-4da4-84ac-716b54f1ef4d","name":"Tame Impala"},"artist":"Tame Impala","track_title":"\\u0027Cause I\\u0027m A Man","track_number":null,"publisher":"BMG","record_label":"FICTION","release_title":null,"record_id":"n3p4fn","catalogue_number":null,"contributions":[{"pid":"p012dc44","name":"Tame Impala","role":"Performer","musicbrainz_gid":"63aa26c3-d59b-4da4-84ac-716b54f1ef4d"}],"title":"\\u0027Cause I\\u0027m A Man","short_synopsis":null,"medium_synopsis":null,"long_synopsis":null}},{"title":null,"pid":"p065l88b","short_synopsis":"","medium_synopsis":"","long_synopsis":"","version_offset":252,"position":2,"is_chapter":false,"has_snippet":false,"segment":{"type":"music","pid":"p062b

In [66]:
response3

<Response [404]>

In [65]:
response3 = requests.get(
#     'https://www.bbc.co.uk/programmes/b09y1wkb/segments.json',
#     'https://www.bbc.co.uk/programmes/b0b11c0n/segments.json',
    'https://www.bbc.co.uk/programmes/m0005l7w/segments.json',
    headers=headers)

In [191]:
res = requests.get(
    'https://www.bbc.co.uk/programmes/b0b11d3n/segments.json',)
#     cookies=my_cookies)

In [192]:
res

<Response [404]>